In [1]:
! pip install beautifulsoup4
! pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import requests
import pandas as pd
import numpy as np
import itertools

In [3]:
def get_articles_urls(company,startpage, endpage):
  urls=[]
  for page in range(startpage, endpage):
    if page % 100 == 0:
      print(page)
    url = f"https://www.investing.com/equities/{company}-inc-news/{page}"
    page=requests.get(url)
    soup=BeautifulSoup(page.text,'html.parser')
    for elt in soup.find_all('div',attrs={'class':'mediumTitle1'})[1].find_all('article'):
        urls.append('https://www.investing.com/'+elt.find('a')['href'])
  return list(itertools.filterfalse(lambda x: x.startswith('https://www.investing.com//pro/offers'), urls))
    

In [4]:
apple_urls=get_articles_urls('apple-computer',4, 1577)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


In [5]:
len(apple_urls)

15599

In [6]:
from time import sleep
def scrape_news(urls, df, company):
  iter = 0
  for url in urls:
    page = requests.get(url)
    soup=BeautifulSoup(page.text,'html.parser')
    if type(soup.find('h1',attrs={'class':'articleHeader'})) is type(None):
      print(url)
      continue
    Title=soup.find('h1',attrs={'class':'articleHeader'}).text.strip()
    Date=soup.find('div',attrs={'class':'contentSectionDetails'}).find("span").text.strip()
    Article=' '.join([x.get_text() for x in soup.find('div',attrs={'class':'WYSIWYG articlePage'}).find_all("p")]).replace('Position added successfully to:','').strip()
    tmpdic = {'ticker': company, 'publish_date': Date, 'title': Title, 'body_text': Article, 'url': url}
    df=df.append(pd.DataFrame(tmpdic, index=[0]))
  return df

In [7]:
articles_df = pd.DataFrame({'ticker':[],
                                'publish_date':[],
                                'title': [],
                                'body_text': [],
                                'url':[]})

In [8]:
apple_df=scrape_news(apple_urls, articles_df, 'Apple')

https://www.investing.com//news/cryptocurrency-news/crypto-flipsider-news--war-news-russia-attacks-nuclear-power-plant-pushinh-btc-and-the-crypto-market-down-again-metamask-blocks-sanctioned-jurisdictions-as-opensea-binance-and-coinbase-take-firm-stance-ukraine-daos-flag-nft-sells
https://www.investing.com//news/cryptocurrency-news/crypto-flipsider-news--terra-raises-1-billion-to-support-ecosystems-growth-tether-reduces-commercial-paper-holdings-metamask-phones-as-hard-wallets-apple-store-hostages-freed-as-gunman-demands-millions-in-crypto-brazils-crypto-re
https://www.investing.com//news/cryptocurrency-news/crypto-flipsider-news--blackrock-to-offer-crypto-trading-voyager-to-allow-usdc-stock-purchase-apple-to-support-crypto-payment-bitcoin-enters-top-10-global-assets-samsung-debuts-s22-in-metaverse-and-mcdonalds-metaverse-restaurant-
https://www.investing.com//news/cryptocurrency-news/crypto-flipsider-news--bitcoin-hits-39k-altcoins-rally-binance-fund-reaches-1-billion-bitfinex-stolen-

In [9]:
apple_df.head()

,ticker,publish_date,title,body_text,url
0,Apple,"Jan 10, 2023 10:30AM ET",Apple to Launch Its Mixed-Reality Headset This...,"After years of development, Apple (NASDAQ:AAPL...",https://www.investing.com//news/cryptocurrency...
0,Apple,"Jan 10, 2023 09:17AM ET","Nasdaq, tech stocks could fail to meaningfully...",By Senad Karaahmetovic Tech stocks are still e...,https://www.investing.com//news/stock-market-n...
0,Apple,"Jan 10, 2023 08:09AM ET","Amazon, Salesforce Job Cuts Are Warning Signs ...",(Bloomberg) -- Tens of thousands of tech secto...,https://www.investing.com//news/stock-market-n...
0,Apple,"Jan 10, 2023 08:06AM ET",Analysts defend Broadcom following Apple socke...,By Senad Karaahmetovic Broadcom (NASDAQ:AVGO) ...,https://www.investing.com//news/stock-market-n...
0,Apple,"Jan 10, 2023 06:56AM ET","Powell speech, Jefferies slump, Brazil arrests...",By Geoffrey Smith Investing.com -- Federal Re...,https://www.investing.com//news/economy/powell...


In [10]:
amazon_urls=get_articles_urls('amazon-com',4,1383)
amazon_df=scrape_news(amazon_urls, articles_df, 'Amazon')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


SSLError: ignored

In [ ]:
amazon_df

In [ ]:
meta_urls=get_articles_urls('facebook',4,1692)
meta_df=scrape_news(meta_urls, articles_df, 'Meta')

In [ ]:
print(len(amazon_urls))
print(len(meta_urls))

In [ ]:
apple_df.to_pickle('apple_news.pkl')
amazon_df.to_pickle('amazon_news.pkl')
meta_df.to_pickle('meta_news.pkl')

In [ ]:
apple_df.to_csv('apple_News')
amazon_df.to_csv('amazon_News')
meta_df.to_csv('meta_News')




# Data Preprocessing

 ***Table of contents***
 

1. Importing Packages and Data Cleaning
2. Visualizations
3. Processing
4. Feature extraction
5. Vader Scores

## Importing Packages & Data Cleaning

### Installing missing libraries and Packages

In [ ]:
pip install emoji

In [ ]:
pip install vaderSentiment

In [ ]:
# Linear Algebra and DataFrames
import numpy as np 
import pandas as pd 

# Plotting
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D

# Visualization libraries
import seaborn as sns
sns.set_style("whitegrid")

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# WordCloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

# NLP Preprocessing and Basic tools
from sklearn.model_selection import train_test_split
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
from emoji import demojize

# Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Models
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.models import Sequential

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Evaluation Metrics
from sklearn.metrics import f1_score, fbeta_score, confusion_matrix

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

### Loading Data

In [ ]:
articles_df.columns


### Data Cleaning & Analysis


In [ ]:
print("Number of articles for Apple:", len(apple_df))
print("Number of articles for Amazon:", len(amazon_df))
print("Number of articles for Meta:", len(meta_df))

### Apple

In [ ]:
apple_df.head()

Duplicates

In [ ]:
# Checking the data for duplicates
apple_df[apple_df.duplicated('body_text',keep=False)].sort_values('body_text')

In [ ]:
# Dropping all duplicates
apple_df.drop_duplicates(('body_text'), inplace=True)

In [ ]:
apple_df.shape

Missing values

In [ ]:
# Missing values
missing = articles_df.isnull().sum().sort_values(ascending = False)
missing.to_frame(name='total').style.background_gradient()

### Amazon

In [ ]:
amazon_df.head()

Duplicates

In [ ]:
# Checking the data for duplicates
amazon_df[amazon_df.duplicated('body_text',keep=False)].sort_values('body_text')

In [ ]:
# Dropping all duplicates
amazon_df.drop_duplicates(('body_text'), inplace=True)

In [ ]:
amazon_df.shape

Missing values

In [ ]:
# Missing values
missing = amazon_df.isnull().sum().sort_values(ascending = False)
missing.to_frame(name='total').style.background_gradient()

### Meta

In [ ]:
meta_df.head()

Duplicates

In [ ]:
# Checking the data for duplicates
meta_df[meta_df.duplicated('body_text',keep=False)].sort_values('body_text')

In [ ]:
# Dropping all duplicates
meta_df.drop_duplicates(('body_text'), inplace=True)

In [ ]:
meta_df.shape

Missing values

In [ ]:
# Missing values
missing = meta_df.isnull().sum().sort_values(ascending = False)
missing.to_frame(name='total').style.background_gradient()

## Visualizations

WordClouds

We added some frequent words used in the description that are related to stock market.

In [ ]:
stopWords = set(STOPWORDS)
stopWords.update(['stock', "market", "NASDAQ", 'Reuters'])


wordcloud = WordCloud(stopwords=stopWords,max_font_size=50,max_words=100,collocations=False, background_color='white').generate(" ".join(text for text in apple_df['body_text']))
plt.figure(figsize=(8,8))
plt.title("Wordcloud for Scrapped news", fontsize=20)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()





As we can see from the wordclouds, there are some neutral words related to economy, and the majority of frequent words doesn't reflect any sentiment.

## Processing

### NLP processing

In [ ]:
def count_tags(articles_df):
  news=articles_df[['body_text','publish_date','title']]
  # Number of mentions, hashtags, urls
  cnt_1, cnt_2, cnt_3 = 0, 0, 0
  max_len, min_len, mean_len = -float("inf"), float("inf"), 0
  for row in news.values:
      text = row[0]  # 0 for text content
      if "@" in text:
          cnt_1 += 1
      if "#" in text:
          cnt_2 += 1
      if 'http' or 'www' in text:
          cnt_3 += 1
      if len(text) < min_len:
          min_len = len(text)
      if len(text) > max_len:
          max_len = len(text)

      mean_len += len(text)
      
  mean_len /= len(articles_df)
      
  print(f'Number of texts having mentions: {cnt_1}')
  print(f'Number of texts having hashtags: {cnt_2}')
  print(f'Number of texts having urls: {cnt_3}')
  print(f'min_len:{min_len} max_len:{max_len} mean_len:{mean_len}')


In [ ]:
print('########### For Apple #####################')
count_tags(apple_df)
print('########### For Amazon #####################')
count_tags(amazon_df)
print('########### For Meta #####################')
count_tags(meta_df)

In [ ]:
from pickle import STOP
# Remove urls
def remove_urls(text):
    return re.sub(r'https?://\S+|www\.\S_+', '', text)

# Remove mentions
def remove_usernames_ressource(text):
    text_split = text.split("-",1)
    if len(text_split)>1:
        text=text_split[1]
    text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
    return text
# Remove hashtags
def remove_hashtags(text):
    return re.sub("#[A-Za-z0-9_]+"," ", text)

# Remove punctuations
punc_list = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punc_list))

# Convert emojis to texts
def convert_emojis(text):
    return demojize(text).replace(":","")

# Apply the previous functions
def full_preprocessing(text):
    """
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove non-ascii words
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # Replace '&amp;' with 'and'
    text = re.sub(r'&amp;', 'and', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    text = remove_urls(text)
    text = remove_usernames_ressource(text)
    text = remove_hashtags(text)
    text = remove_punctuation(text)
    text = convert_emojis(text)

    return text.lower()

# Trasform text to tokens (separated words)
def tokenize(text) :
    text = text.split()
    return text

# Remove stopwords
stop_words = stopwords.words('english')
def remove_stopwords(text):
    words_to_keep = ["not","no","nor"]
    stopword = [elem for elem in stop_words if not elem in words_to_keep]
    text = [w.lower() for w in text if not w.lower() in stopword]
    return text

# Lemmatization
wn = nltk.WordNetLemmatizer()
def lemmatize(text):
    text = [wn.lemmatize(word) for word in text]
    return text

# Stemming
ps = nltk.PorterStemmer()
ls = nltk.LancasterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    text = [ls.stem(word) for word in text]
    return text



In [ ]:
def full_processing(df):
    df["text_W_puncts"] =df["body_text"].apply(lambda x: full_preprocessing(x))
    df["text_tokenized"] = df["text_W_puncts"].apply(lambda x: tokenize(x))
    df["text_W_stopwords"] = df["text_tokenized"].apply(lambda x: remove_stopwords(x))
    df["text_lemmatized"] = df["text_W_stopwords"].apply(lambda x: lemmatize(x))
    df["text_stemmed"] = df["text_lemmatized"].apply(lambda x: stemming(x))
    df["text_processed"] = df["text_stemmed"].apply(lambda x: ' '.join(str(e) for e in x))
    
    return df
    

In [ ]:
apple_processed = full_processing(apple_df)
amazon_processed = full_processing(amazon_df)
meta_processed = full_processing(meta_df)


In [ ]:
apple_processed

# Vader Scores

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# init the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def predicted_label(x):
  if x<=-0.5:
    return 0
  elif x>=0.5:
    return 2
  else:
    return 1

def score_Vader(df,analyzer):
  df['neg'] = df['body_text'].apply(lambda x:analyzer.polarity_scores(x)['neg'])
  df['neu'] = df['body_text'].apply(lambda x:analyzer.polarity_scores(x)['neu'])
  df['pos'] = df['body_text'].apply(lambda x:analyzer.polarity_scores(x)['pos'])
  df['compound'] = df['body_text'].apply(lambda x:analyzer.polarity_scores(x)['compound'])
  
  df['predicted_class'] = df['compound'].map(predicted_label) 
  return df

In [ ]:
apple_withScore=score_Vader(apple_processed, analyzer)
amazon_withScore=score_Vader(amazon_processed, analyzer)
meta_withScore=score_Vader(meta_processed, analyzer)

In [ ]:
apple_withScore

In [ ]:
apple_withScore.to_pickle("apple_news_sentiments.pkl")
amazon_withScore.to_pickle("amazon_news_sentiments.pkl")
meta_withScore.to_pickle("meta_news_sentiments.pkl")

# Hopsworks

In [ ]:
!pip install hopsworks

In [ ]:
!pip install python-dotenv

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

In [ ]:
from datetime import datetime

def timestamp_2_time(x):  
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

def process_event_date(articles_df):
  articles_df['publish_date'] = pd.to_datetime(articles_df['publish_date'], format='%b %d, %Y %I:%M%p ET')
  articles_df.publish_date = articles_df.publish_date.apply(timestamp_2_time)
  return articles_df


In [ ]:
apple_news_df=process_event_date(apple_withScore)
amazon_news_df=process_event_date(amazon_withScore)
meta_news_df=process_event_date(meta_withScore)

In [ ]:
frames = [apple_news_df, amazon_news_df, meta_news_df]
articles_df = pd.concat(frames)
articles_df

In [ ]:
fs = project.get_feature_store()

market_news_fg = fs.get_or_create_feature_group(
        name = 'market_news_fg_for_three',
        description = 'Market news for Apple, Amazon, and Meta',
        version = 1,
        primary_key = ['publish_date','ticker'],
        online_enabled = True,
        event_time = 'publish_date'
    )    

market_news_fg.insert(articles_df)